In [1]:
# Need this for this demo to even be possible,
# turns out that jupyter already runs in its own
#  event loop which is NOT pretty to deal with
import nest_asyncio
nest_asyncio.apply()
import asyncio

%load_ext memory_profiler

In [2]:
import requests

print(requests.get("http://0.0.0.0:8080").text)
input()
print(requests.get("http://0.0.0.0:8080/items").text)
input()
print(requests.get("http://0.0.0.0:8080/items/1").text)

Hello, world

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
{"email": "mooreryan@example.net"}


In [3]:
for _ in range(70):
    print(requests.get("http://0.0.0.0:8080/items/1").text)

{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan@example.net"}
{"email": "mooreryan

In [4]:
from aiohttp import request
import asyncio

async def req(i):
    async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
        print(await r.content.read())

asyncio.run(req(1))

b'{"email": "mooreryan@example.net"}'


In [5]:
import asyncio
import time

class ThrottledQueue(asyncio.Queue):
    "subclass asyncio.Queue i.e. import all behaviour"

    def __init__(self, per_second, maxsize=0, *, loop=None, i=0):
        "Set up some extra vars and then call the original init"

        self.lock = asyncio.Lock()
        self.i = i
        self.per_second = per_second
        self.last_get = time.time() # this is the fastest way... I think?
        super(ThrottledQueue, self).__init__(maxsize=maxsize, loop=loop)

    async def retry(self):
        """
        Signals to the queue that an item is being retried, 
        so pause any get()s by aquiring the lock and throttling before releasing
        """
        async with self.lock:
            await self._throttle()

    async def get(self):
        async with self.lock:
            await self._throttle()
            result = await super(ThrottledQueue, self).get()

            self.last_get = time.time()
            return result

    async def _throttle(self):
        elapsed = time.time() - self.last_get
        sleep_time = self.per_second - elapsed
        print(self.i, '- times', f'{elapsed:.5f}', f'{sleep_time:.5f}', '- sizes', self.qsize(), f'{self.qsize() / self.maxsize:.5f}')
        await asyncio.sleep(max(0, sleep_time)) # Make sure we wait at least 0 seconds


In [6]:
from throttler import Throttler

async def req(i, t: Throttler):
    print(i, "starting")
    async with t:
        print(i, "inside throttler")
        async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
            print(i, "before request")
            resp = await r.content.read()
            print(i, "after request")
            return resp

t = Throttler(rate_limit=60, period=10.0)
results = asyncio.run(asyncio.gather(*[asyncio.create_task(req(i, t)) for i in range(70)]))

0 starting
0 inside throttler
1 starting
1 inside throttler
2 starting
2 inside throttler
3 starting
3 inside throttler
4 starting
4 inside throttler
5 starting
5 inside throttler
6 starting
6 inside throttler
7 starting
7 inside throttler
8 starting
8 inside throttler
9 starting
9 inside throttler
10 starting
10 inside throttler
11 starting
11 inside throttler
12 starting
12 inside throttler
13 starting
13 inside throttler
14 starting
14 inside throttler
15 starting
15 inside throttler
16 starting
16 inside throttler
17 starting
17 inside throttler
18 starting
18 inside throttler
19 starting
19 inside throttler
20 starting
20 inside throttler
21 starting
21 inside throttler
22 starting
22 inside throttler
23 starting
23 inside throttler
24 starting
24 inside throttler
25 starting
25 inside throttler
26 starting
26 inside throttler
27 starting
27 inside throttler
28 starting
28 inside throttler
29 starting
29 inside throttler
30 starting
30 inside throttler
31 starting
31 inside thrott

In [7]:
from tenacity import retry, stop_after_attempt, before_log

from throttler import Throttler

import logging

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)

# @retry(stop=stop_after_attempt(3), before=before_log(logger, logging.DEBUG))
@retry(stop=stop_after_attempt(3))
async def req(i, t: Throttler):
    async with t:
        async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
            resp = await r.content.read()
            r.raise_for_status()
            return resp

def run_reqs(rate_limit: int, period: float, n_reqs: int):
    t = Throttler(rate_limit=rate_limit, period=period)
    asyncio.run(asyncio.gather(*[asyncio.create_task(req(i, t)) for i in range(n_reqs)]))

def run_reqs_2(rate_limit: int, period: float, n_reqs: int):
    t = Throttler(rate_limit=rate_limit, period=period)
    tasks = [req(i, t) for i in range(n_reqs)]

async def testr(n):
    return n

def create_tasks(n):
    tasks = [asyncio.create_task(testr(i)) for i in range(n)]

def create_tasks_2(n):
    tasks = [testr(i) for i in range(n)]

In [8]:
# %memit run_reqs(10, 1.0, 1000)
%memit run_reqs_2(10, 1.0, 1000)
# %memit run_reqs_3(10, 1.0, 1000)
%memit create_tasks(1000)

<string>:1: RuntimeWarning: coroutine 'req' was never awaited


peak memory: 70.79 MiB, increment: 0.29 MiB
